In [1]:
import numpy as np
import pandas as pd
import sys
sys.path.insert(1, '../')
import scripts.src as src
import dimod

In [3]:
player_data = pd.read_csv('../data/player_data.csv')
del player_data['Unnamed: 0']
all_points = pd.read_csv('../data/all_points.csv')
del all_points['Unnamed: 0']

In [4]:
players = player_data[['name','position']].drop_duplicates(subset='name')

players['costs'] = players.apply(src.cost_filler, axis=1)
players[['CD','CF','CG']] = pd.DataFrame(players.costs.tolist(), 
                                          index= players.index)
del players['costs']
del players['position']
players = players.set_index('name')
players

,CD,CF,CG
name,,,
Adam Boqvist,1,0,0
Adam Gaudette,0,1,0
Adam Pelech,1,0,0
Adam Werner,0,0,1
Adin Hill,0,0,1
...,...,...,...
Yanni Gourde,0,1,0
Zach Bogosian,1,0,0
Zach Werenski,1,0,0


In [5]:

num_each = {'CD':2, 'CF':4, 'CG': 2}
rets = all_points.mean().head(50)
cov  = all_points[list(rets.keys())].cov()
    
lins = src.linear_terms(rets, players, num_each, sign = -1)
quads = src.quadratic_terms(cov, players)


In [6]:
%%time
import dimod 
bqm = dimod.BQM(lins, 
                quads,
                0,
                dimod.BINARY)

test = dimod.SimulatedAnnealingSampler().sample(bqm)

CPU times: user 17.6 s, sys: 11.7 ms, total: 17.6 s
Wall time: 17.6 s


In [7]:
choices = list(test.first.sample.items())
filt = []
for name in choices:
    if name[1] == 1: 
        filt.append(name[0])
player_data[player_data.name.isin(filt)][['name', 'position']].drop_duplicates(subset = 'name')

,name,position
306,Adam Pelech,D
612,Adin Hill,G
1634,Alex Pietrangelo,D
2948,Anders Lee,L
3407,Andrei Vasilevskiy,G
4882,Artemi Panarin,L
5442,Bo Horvat,C
5761,Brad Marchand,L
